Ön hazırlık (kütüphane kurulumu)

In [1]:
# Gerekli kütüphaneleri yükleyelim (Colab hücresine)
!pip install cirq qiskit qutip numpy matplotlib --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.8/670.8 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 430.5/430.5 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 85.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.8/31.8 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 70.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 73.1 MB/s eta 0:00:00


BB84 PROTOKOLÜ

Cirq ile BB84 Örneği

In [2]:
import cirq
import numpy as np

# Alice ve Bob için qubit sayısı
n = 10
qubits = cirq.LineQubit.range(n)

# Alice'in rastgele bit ve baz seçimleri
alice_bits = np.random.randint(2, size=n)
alice_bases = np.random.choice(['Z', 'X'], size=n)  # Z=basis, X=Hadamard

# Bob'un rastgele baz seçimleri
bob_bases = np.random.choice(['Z', 'X'], size=n)

# Devreyi oluştur
circuit = cirq.Circuit()

for i in range(n):
    # Alice bitini uygula
    if alice_bits[i] == 1:
        circuit.append(cirq.X(qubits[i]))
    # Alice bazını uygula
    if alice_bases[i] == 'X':
        circuit.append(cirq.H(qubits[i]))
    # Bob bazını uygula
    if bob_bases[i] == 'X':
        circuit.append(cirq.H(qubits[i]))
    # Ölçüm
    circuit.append(cirq.measure(qubits[i], key=f'q{i}'))

print("BB84 Devresi:")
print(circuit)

# Simülasyon
simulator = cirq.Simulator()
result = simulator.run(circuit)

# Ölçüm sonuçlarını al
bob_results = [result.measurements[f'q{i}'][0][0] for i in range(n)]

# Ortak anahtarı belirle (sadece aynı bazları kullandılar)
shared_key = [alice_bits[i] for i in range(n) if alice_bases[i] == bob_bases[i]]
print("\nAlice'in bitleri:", alice_bits)
print("Alice bazları:", alice_bases)
print("Bob bazları:", bob_bases)
print("Bob ölçüm sonuçları:", bob_results)
print("Oluşan ortak anahtar:", shared_key)


BB84 Devresi:
0: ───X─────────H─────────H─────────M('q0')───

1: ───H─────────M('q1')───────────────────────

2: ───X─────────H─────────H─────────M('q2')───

3: ───X─────────H─────────M('q3')─────────────

4: ───H─────────M('q4')───────────────────────

5: ───X─────────H─────────M('q5')─────────────

6: ───M('q6')─────────────────────────────────

7: ───X─────────H─────────H─────────M('q7')───

8: ───X─────────H─────────H─────────M('q8')───

9: ───H─────────M('q9')───────────────────────

Alice'in bitleri: [1 0 1 1 0 1 0 1 1 0]
Alice bazları: ['X' 'Z' 'X' 'X' 'X' 'X' 'Z' 'X' 'X' 'Z']
Bob bazları: ['X' 'X' 'X' 'Z' 'Z' 'Z' 'Z' 'X' 'X' 'X']
Bob ölçüm sonuçları: [np.int8(1), np.int8(1), np.int8(1), np.int8(0), np.int8(1), np.int8(1), np.int8(0), np.int8(1), np.int8(1), np.int8(0)]
Oluşan ortak anahtar: [np.int64(1), np.int64(1), np.int64(0), np.int64(1), np.int64(1)]


Qiskit ile BB84 Örneği

In [3]:
from qiskit import QuantumCircuit, Aer, execute
import numpy as np

n = 10

# Alice bit ve baz seçimleri
alice_bits = np.random.randint(2, size=n)
alice_bases = np.random.choice(['Z','X'], size=n)

# Bob bazları
bob_bases = np.random.choice(['Z','X'], size=n)

qc = QuantumCircuit(n, n)

for i in range(n):
    # Alice bit
    if alice_bits[i] == 1:
        qc.x(i)
    # Alice baz
    if alice_bases[i] == 'X':
        qc.h(i)
    # Bob baz
    if bob_bases[i] == 'X':
        qc.h(i)
    qc.measure(i, i)

print("BB84 Quantum Circuit:")
print(qc.draw(output='text'))

# Simülasyon
backend = Aer.get_backend('qasm_simulator')
job = execute(qc, backend, shots=1)
result = job.result()
counts = result.get_counts()

# Ölçüm sonucu
bitstring = list(counts.keys())[0][::-1]  # Qiskit bit sıralaması ters
shared_key = [alice_bits[i] for i in range(n) if alice_bases[i] == bob_bases[i]]

print("\nAlice bitleri:", alice_bits)
print("Alice bazları:", alice_bases)
print("Bob bazları:", bob_bases)
print("Bob ölçüm sonucu:", bitstring)
print("Oluşan ortak anahtar:", shared_key)


ImportError: cannot import name 'Aer' from 'qiskit' (/usr/local/lib/python3.12/dist-packages/qiskit/__init__.py)

QuTiP ile BB84 Örneği

In [4]:
import numpy as np
from qutip import basis, tensor, hadamard_transform, measurement_statistics

n = 10

# Alice bit ve baz
alice_bits = np.random.randint(2, size=n)
alice_bases = np.random.choice(['Z','X'], size=n)
bob_bases = np.random.choice(['Z','X'], size=n)

# Qubitleri oluştur
alice_qubits = []
for i in range(n):
    qubit = basis(2, alice_bits[i])
    if alice_bases[i] == 'X':
        qubit = hadamard_transform() * qubit
    alice_qubits.append(qubit)

# Bob ölçümleri
bob_results = []
for i in range(n):
    qubit = alice_qubits[i]
    if bob_bases[i] == 'X':
        qubit = hadamard_transform() * qubit
    # Ölçüm
    p0, p1 = measurement_statistics(qubit, [basis(2,0), basis(2,1)])
    bob_results.append(np.random.choice([0,1], p=[p0,p1]))

# Ortak anahtar
shared_key = [alice_bits[i] for i in range(n) if alice_bases[i]==bob_bases[i]]

print("Alice bitleri:", alice_bits)
print("Alice bazları:", alice_bases)
print("Bob bazları:", bob_bases)
print("Bob ölçüm sonucu:", bob_results)
print("Oluşan ortak anahtar:", shared_key)


ImportError: cannot import name 'hadamard_transform' from 'qutip' (/usr/local/lib/python3.12/dist-packages/qutip/__init__.py)

B92 PROTOKOLÜ

Cirq ile B92 Örneği

In [5]:
import cirq
import numpy as np

# Qubit sayısı
n = 8
qubits = cirq.LineQubit.range(n)

# Alice'in bitleri (sadece 0 veya 0' bazları)
alice_bits = np.zeros(n, dtype=int)  # tüm bitler 0
# Alice'in bazları: 0 -> |0>, 0' -> |0'>
alice_bases = np.random.choice(['0', '0prime'], size=n)

# Bob'un ölçüm bazları: hesaplama bazı veya dik olmayan baz
bob_bases = np.random.choice(['0_1', '0prime_1prime'], size=n)

circuit = cirq.Circuit()

# Alice'in qubitlerini hazırla
for i in range(n):
    if alice_bases[i] == '0prime':
        circuit.append(cirq.ry(np.pi/4)(qubits[i]))  # dik olmayan baz dönüşümü
    # Bob'un bazını uygula
    if bob_bases[i] == '0prime_1prime':
        circuit.append(cirq.ry(-np.pi/4)(qubits[i]))
    # Ölçüm
    circuit.append(cirq.measure(qubits[i], key=f'q{i}'))

simulator = cirq.Simulator()
result = simulator.run(circuit)

# Ölçüm sonuçları
bob_results = [result.measurements[f'q{i}'][0][0] for i in range(n)]

# Bob'un anahtar oluşturma (ölçüm sonucu 1 veya 1' ise)
shared_key_indices = [i for i in range(n) if bob_results[i] == 1]
shared_key = [bob_results[i] for i in shared_key_indices]

print("Alice bazları:", alice_bases)
print("Bob bazları:", bob_bases)
print("Bob ölçüm sonuçları:", bob_results)
print("Anahtar için seçilen pozisyonlar:", shared_key_indices)
print("Oluşan anahtar:", shared_key)


Alice bazları: ['0prime' '0prime' '0prime' '0' '0prime' '0prime' '0prime' '0prime']
Bob bazları: ['0prime_1prime' '0_1' '0_1' '0_1' '0prime_1prime' '0_1' '0prime_1prime'
 '0_1']
Bob ölçüm sonuçları: [np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0), np.int8(0)]
Anahtar için seçilen pozisyonlar: []
Oluşan anahtar: []


Qiskit ile B92 Örneği

In [6]:
from qiskit import QuantumCircuit, Aer, execute
import numpy as np

n = 8

# Alice bazları
alice_bases = np.random.choice(['0','0prime'], size=n)
# Bob bazları
bob_bases = np.random.choice(['0_1','0prime_1prime'], size=n)

qc = QuantumCircuit(n, n)

for i in range(n):
    # Alice qubit
    if alice_bases[i] == '0prime':
        qc.ry(np.pi/4, i)
    # Bob bazı
    if bob_bases[i] == '0prime_1prime':
        qc.ry(-np.pi/4, i)
    qc.measure(i, i)

backend = Aer.get_backend('qasm_simulator')
job = execute(qc, backend, shots=1)
result = job.result()
bitstring = list(result.get_counts().keys())[0][::-1]

# Bob anahtarı oluştur
bob_results = [int(b) for b in bitstring]
shared_key_indices = [i for i in range(n) if bob_results[i] == 1]
shared_key = [bob_results[i] for i in shared_key_indices]

print("Alice bazları:", alice_bases)
print("Bob bazları:", bob_bases)
print("Bob ölçüm sonucu:", bob_results)
print("Anahtar için seçilen pozisyonlar:", shared_key_indices)
print("Oluşan anahtar:", shared_key)


ImportError: cannot import name 'Aer' from 'qiskit' (/usr/local/lib/python3.12/dist-packages/qiskit/__init__.py)

QuTiP ile B92 Örneği

In [7]:
import numpy as np
from qutip import basis, ket, Qobj, measurement_statistics, ry

n = 8

# Alice bazları ve Bob bazları
alice_bases = np.random.choice(['0','0prime'], size=n)
bob_bases = np.random.choice(['0_1','0prime_1prime'], size=n)

# Alice qubitleri
alice_qubits = []
for i in range(n):
    qubit = basis(2,0)  # |0>
    if alice_bases[i] == '0prime':
        qubit = ry(np.pi/4) * qubit
    alice_qubits.append(qubit)

# Bob ölçümleri
bob_results = []
for i in range(n):
    qubit = alice_qubits[i]
    if bob_bases[i] == '0prime_1prime':
        qubit = ry(-np.pi/4) * qubit
    p0, p1 = measurement_statistics(qubit, [basis(2,0), basis(2,1)])
    bob_results.append(np.random.choice([0,1], p=[p0,p1]))

# Anahtar oluşturma
shared_key_indices = [i for i in range(n) if bob_results[i]==1]
shared_key = [bob_results[i] for i in shared_key_indices]

print("Alice bazları:", alice_bases)
print("Bob bazları:", bob_bases)
print("Bob ölçüm sonuçları:", bob_results)
print("Anahtar için seçilen pozisyonlar:", shared_key_indices)
print("Oluşan anahtar:", shared_key)


ImportError: cannot import name 'measurement_statistics' from 'qutip' (/usr/local/lib/python3.12/dist-packages/qutip/__init__.py)

E91 PROTOKOLÜ

Cirq ile E91 Örneği

In [8]:
import cirq
import numpy as np

# Qubit sayısı (Alice ve Bob için çiftler)
n = 4
alice_qubits = cirq.LineQubit.range(n)
bob_qubits = cirq.LineQubit.range(n, 2*n)

circuit = cirq.Circuit()

# Bell durumlarını hazırla (|Φ+> = (|00> + |11>)/√2)
for i in range(n):
    circuit.append(cirq.H(alice_qubits[i]))
    circuit.append(cirq.CNOT(alice_qubits[i], bob_qubits[i]))

# Alice ve Bob rastgele bazlar seçer
alice_bases = np.random.choice(['Z','X'], size=n)
bob_bases = np.random.choice(['Z','X'], size=n)

# Alice ölçüm bazını uygula
for i in range(n):
    if alice_bases[i] == 'X':
        circuit.append(cirq.H(alice_qubits[i]))
    if bob_bases[i] == 'X':
        circuit.append(cirq.H(bob_qubits[i]))

# Ölçümler
for i in range(n):
    circuit.append(cirq.measure(alice_qubits[i], key=f'A{i}'))
    circuit.append(cirq.measure(bob_qubits[i], key=f'B{i}'))

print("E91 Devresi:")
print(circuit)

simulator = cirq.Simulator()
result = simulator.run(circuit)

# Ölçüm sonuçları
alice_results = [result.measurements[f'A{i}'][0][0] for i in range(n)]
bob_results = [result.measurements[f'B{i}'][0][0] for i in range(n)]

# Ortak anahtar: Alice ve Bob aynı bazda ölçüm yaptıysa
shared_key = [alice_results[i] for i in range(n) if alice_bases[i]==bob_bases[i]]

print("\nAlice bazları:", alice_bases)
print("Bob bazları:", bob_bases)
print("Alice ölçüm sonuçları:", alice_results)
print("Bob ölçüm sonuçları:", bob_results)
print("Oluşan ortak anahtar:", shared_key)


E91 Devresi:
          ┌────┐
0: ───H────@───────M('A0')─────────────
           │
1: ───H────┼@──────M('A1')─────────────
           ││
2: ───H────┼┼@─────H─────────M('A2')───
           │││
3: ───H────┼┼┼@────M('A3')─────────────
           ││││
4: ────────X┼┼┼────H─────────M('B0')───
            │││
5: ─────────X┼┼────M('B1')─────────────
             ││
6: ──────────X┼────H─────────M('B2')───
              │
7: ───────────X────M('B3')─────────────
          └────┘

Alice bazları: ['Z' 'Z' 'X' 'Z']
Bob bazları: ['X' 'Z' 'X' 'Z']
Alice ölçüm sonuçları: [np.int8(0), np.int8(1), np.int8(0), np.int8(0)]
Bob ölçüm sonuçları: [np.int8(1), np.int8(1), np.int8(0), np.int8(0)]
Oluşan ortak anahtar: [np.int8(1), np.int8(0), np.int8(0)]


Qiskit ile E91 Örneği

In [9]:
from qiskit import QuantumCircuit, Aer, execute
import numpy as np

n = 4
qc = QuantumCircuit(2*n, 2*n)

# Bell durumlarını hazırla
for i in range(n):
    qc.h(i)
    qc.cx(i, i+n)

# Alice ve Bob bazları
alice_bases = np.random.choice(['Z','X'], size=n)
bob_bases = np.random.choice(['Z','X'], size=n)

# Baz dönüşümleri
for i in range(n):
    if alice_bases[i] == 'X':
        qc.h(i)
    if bob_bases[i] == 'X':
        qc.h(i+n)

# Ölçüm
for i in range(n):
    qc.measure(i, i)
    qc.measure(i+n, i+n)

backend = Aer.get_backend('qasm_simulator')
job = execute(qc, backend, shots=1)
result = job.result()
bitstring = list(result.get_counts().keys())[0][::-1]

alice_results = [int(bitstring[i]) for i in range(n)]
bob_results = [int(bitstring[i+n]) for i in range(n)]

shared_key = [alice_results[i] for i in range(n) if alice_bases[i]==bob_bases[i]]

print("Alice bazları:", alice_bases)
print("Bob bazları:", bob_bases)
print("Alice ölçüm sonuçları:", alice_results)
print("Bob ölçüm sonuçları:", bob_results)
print("Oluşan ortak anahtar:", shared_key)


ImportError: cannot import name 'Aer' from 'qiskit' (/usr/local/lib/python3.12/dist-packages/qiskit/__init__.py)

QuTiP ile E91 Örneği

In [10]:
import numpy as np
from qutip import basis, tensor, measurement_statistics, hadamard_transform

n = 4

# Bell durumlarını hazırla |Φ+>
bell_states = [tensor(basis(2,0), basis(2,0)) + tensor(basis(2,1), basis(2,1)) for _ in range(n)]
bell_states = [state.unit() for state in bell_states]  # Normalize et

# Alice ve Bob bazları
alice_bases = np.random.choice(['Z','X'], size=n)
bob_bases = np.random.choice(['Z','X'], size=n)

alice_results = []
bob_results = []

for i in range(n):
    qubit = bell_states[i]

    # Alice baz dönüşümü
    if alice_bases[i] == 'X':
        qubit = tensor(hadamard_transform(), np.identity(2)) * qubit
    # Bob baz dönüşümü
    if bob_bases[i] == 'X':
        qubit = tensor(np.identity(2), hadamard_transform()) * qubit

    # Ölçüm
    p00, p01, p10, p11 = [np.abs((tensor(basis(2,i), basis(2,j)).dag() * qubit).full()[0][0])**2 for i in [0,1] for j in [0,1]]
    outcomes = [0,0,1,1]  # mantıksal bitler
    probs = [p00, p01, p10, p11]
    choice = np.random.choice([0,1,2,3], p=probs/np.sum(probs))

    alice_bit = 0 if choice in [0,1] else 1
    bob_bit = 0 if choice in [0,2] else 1

    alice_results.append(alice_bit)
    bob_results.append(bob_bit)

shared_key = [alice_results[i] for i in range(n) if alice_bases[i]==bob_bases[i]]

print("Alice bazları:", alice_bases)
print("Bob bazları:", bob_bases)
print("Alice ölçüm sonuçları:", alice_results)
print("Bob ölçüm sonuçları:", bob_results)
print("Oluşan ortak anahtar:", shared_key)


ImportError: cannot import name 'measurement_statistics' from 'qutip' (/usr/local/lib/python3.12/dist-packages/qutip/__init__.py)

DEUTSCH-JOZSA ALGORİTMASI İLE NASIL ATAK YAPILIR?

Cirq ile Deutsch-Jozsa + Entanglement Attack

In [12]:
import cirq
import numpy as np

n = 3  # Giriş qubit sayısı

# Alice qubitleri
alice_qubits = cirq.LineQubit.range(n)
# Son qubit
ancilla = cirq.LineQubit(n)
# Saldırgan qubitleri
eve_qubits = cirq.LineQubit(range(n+1, n+1+n))

circuit = cirq.Circuit()

# Başlangıç durumları
circuit.append([cirq.X(ancilla)])  # |1> durumuna getir
circuit.append([cirq.H(q) for q in alice_qubits + [ancilla]])

# Saldırganın dolanıklık saldırısı: A_i qubitini kendi qubit ile Bell durumu yap
for i in range(n):
    circuit.append(cirq.H(alice_qubits[i]))
    circuit.append(cirq.CNOT(alice_qubits[i], eve_qubits[i]))

# Karakutu U_f (sabit fonksiyon örneği: f(x)=0)
# U_f: y -> y ⊕ f(x), burada sabit 0 için sadece identite
# Bu örnekte sadece göstermek için pas geçiyoruz

# Hadamard tekrar
circuit.append([cirq.H(q) for q in alice_qubits + [ancilla]])
circuit.append([cirq.measure(q, key=f'A{i}') for i,q in enumerate(alice_qubits)])
circuit.append(cirq.measure(ancilla, key='B'))

sim = cirq.Simulator()
result = sim.run(circuit)

print("Ölçüm sonuçları (Alice ve Ancilla):")
print(result)


TypeError: 'LineQubit' object is not subscriptable

Qiskit ile Deutsch-Jozsa + Entanglement Attack

In [13]:
from qiskit import QuantumCircuit, Aer, execute

n = 3
qc = QuantumCircuit(2*n + 1, n + 1)  # Alice, Eve ve ancilla

# Başlangıç durumları
qc.x(n*2)  # ancilla |1>
qc.h(range(n))  # Alice qubitleri
qc.h(n*2)      # ancilla

# Eve ile dolanıklık saldırısı
for i in range(n):
    qc.h(i)
    qc.cx(i, n + i)  # A_i ile Eve qubitleri dolanık

# Karakutu U_f (f(x)=0)
# Örnek: pas geçiyoruz

# Hadamard tekrar
qc.h(range(n))
qc.h(n*2)

# Ölçüm
qc.measure(range(n), range(n))
qc.measure(n*2, n)

backend = Aer.get_backend('qasm_simulator')
job = execute(qc, backend, shots=1)
result = job.result()
counts = result.get_counts()

print("Ölçüm sonuçları:")
print(counts)


ImportError: cannot import name 'Aer' from 'qiskit' (/usr/local/lib/python3.12/dist-packages/qiskit/__init__.py)

QuTiP ile Deutsch-Jozsa + Entanglement Attack

In [14]:
import numpy as np
from qutip import basis, tensor, hadamard_transform, qeye, measurement_statistics

n = 3
# Başlangıç durumları |0001>
alice = tensor([basis(2,0) for _ in range(n)])
ancilla = basis(2,1)
eve = tensor([basis(2,0) for _ in range(n)])

# Dolanıklık saldırısı: |phi+> = (|00> + |11>)/√2
bell = (tensor(basis(2,0), basis(2,0)) + tensor(basis(2,1), basis(2,1))).unit()
# Her Alice qubit ile Eve qubiti dolanık
state = tensor(bell, bell, basis(2,1))  # Örnek n=2 için

# Hadamard tüm Alice ve ancilla
Hn = hadamard_transform()
# Ölçüm örneği: sadece göstermek için
# QuTiP ölçümü simülasyonu burada yapılabilir

print("Deutsch-Jozsa başlangıç durumu (saldırgan dolanıklı):")
print(state)


ImportError: cannot import name 'hadamard_transform' from 'qutip' (/usr/local/lib/python3.12/dist-packages/qutip/__init__.py)

SIMON ALGORİTMASI İLE NASIL ATAK YAPILIR?  

Cirq Örneği

In [15]:
import cirq
import numpy as np

# Qubitleri oluştur
n = 2  # örnek olarak 2-bit Simon
A = [cirq.LineQubit(i) for i in range(n)]
C = [cirq.LineQubit(i+n) for i in range(n)]
B = cirq.LineQubit(2*n)

# Simon fonksiyonu (örnek s = 11)
s = np.array([1,1])
f_map = { (0,0):(0,0), (0,1):(0,1), (1,0):(1,1), (1,1):(1,0) }

def simon_U(circuit, qubits_A, qubit_B):
    for x in [(0,0),(0,1),(1,0),(1,1)]:
        fx = f_map[x]
        # basit simülasyon olarak kontrollü X uygulanabilir
        for i, bit in enumerate(fx):
            if bit:
                circuit.append(cirq.X(qubit_B).controlled_by(*[qubits_A[j] for j, b in enumerate(x) if b]))

# Devre oluştur
circuit = cirq.Circuit()

# Başlangıç durumu ve dolanıklık saldırısı
for i in range(n):
    circuit.append(cirq.H(A[i]))
    circuit.append(cirq.H(C[i]))
    # Dolanık hale getirme
    circuit.append(cirq.CNOT(A[i], C[i]))

# B qubitini başlat
circuit.append(cirq.X(B))
circuit.append(cirq.H(B))

# Karakutu uygulama
simon_U(circuit, A, B)

# Hadamard tekrar
for q in A + C:
    circuit.append(cirq.H(q))

# Ölçümler
circuit.append([cirq.measure(q, key=f'a{i}') for i, q in enumerate(A)])
circuit.append([cirq.measure(q, key=f'c{i}') for i, q in enumerate(C)])
circuit.append(cirq.measure(B, key='B'))

simulator = cirq.Simulator()
result = simulator.run(circuit, repetitions=10)
print(result)


B=0011010011
a0=1000111111
a1=1001001101
c0=0000000000
c1=0000000000


Qiskit Örneği

In [16]:
from qiskit import QuantumCircuit, Aer, execute
from qiskit.visualization import plot_histogram

n = 2
A = [i for i in range(n)]
C = [i+n for i in range(n)]
B = 2*n

qc = QuantumCircuit(2*n+1, 2*n+1)

# Başlangıç ve dolanıklık
for i in range(n):
    qc.h(i)      # A qubitleri
    qc.h(i+n)    # C qubitleri
    qc.cx(i, i+n) # Dolanıklık

qc.x(B)
qc.h(B)

# Basit simon fonksiyonu (örnek)
# Kontrollü-X uygulanabilir
qc.cx(0,B)
qc.cx(1,B)

# Hadamard tekrar
for i in range(n):
    qc.h(i)
    qc.h(i+n)

# Ölçüm
for i in range(2*n+1):
    qc.measure(i, i)

backend = Aer.get_backend('qasm_simulator')
result = execute(qc, backend, shots=10).result()
counts = result.get_counts()
print(counts)


ImportError: cannot import name 'Aer' from 'qiskit' (/usr/local/lib/python3.12/dist-packages/qiskit/__init__.py)

QuTiP Örneği

In [17]:
from qutip import *
import numpy as np

n = 2

# Basis state oluştur
A = [basis(2,0) for _ in range(n)]
C = [basis(2,0) for _ in range(n)]
B = basis(2,1)  # B qubitini |1> olarak başlat

# Tensor ürünleri
state = tensor(A + C + [B])

# Hadamard operatörü
H = (1/np.sqrt(2)) * Qobj([[1,1],[1,-1]])

# Dolanıklık: CNOT
def apply_cnot(state, control, target, n_total):
    X = sigmax()
    P0 = basis(2,0)*basis(2,0).dag()
    P1 = basis(2,1)*basis(2,1).dag()
    op = tensor([P0 if i==control else qeye(2) for i in range(n_total)]) + \
         tensor([P1 if i==control else (X if i==target else qeye(2)) for i in range(n_total)])
    return op*state

# Hadamard A ve C
for i in range(n):
    H_i = tensor([H if j==i or j==i+n else qeye(2) for j in range(2*n+1)])
    state = H_i*state
    state = apply_cnot(state, i, i+n, 2*n+1)

# B qubitini Hadamard ve X
state = tensor([qeye(2) for _ in range(2*n)] + [sigmax()*H*B]) * state

print(state)


TypeError: incompatible dimensions [[2, 2, 2, 2, 2], [2, 2, 2, 2, 1]] and [[2, 2, 2, 2, 2], [1]]

GROVER ALGORİTMASI İLE NASIL ATAK YAPILIR?

Cirq Örneği (2 qubit Grover + dolanıklık)

In [18]:
import cirq
import numpy as np

# Qubitler
A = [cirq.LineQubit(i) for i in range(2)]
B = [cirq.LineQubit(i+2) for i in range(2)]

# Devre
circuit = cirq.Circuit()

# Başlangıç durumu ve dolanıklık
for i in range(2):
    circuit.append(cirq.H(A[i]))
    circuit.append(cirq.H(B[i]))
    circuit.append(cirq.CNOT(A[i], B[i]))  # Dolanıklık

# Oracle fonksiyonu örneği (aranan state |00>)
circuit.append(cirq.Z(A[0]).controlled_by(*A[1:]))  # Basit oracle
circuit.append(cirq.Z(A[1]).controlled_by(*A[:1]))

# Diffusion operator (Grover iterasyonu)
for q in A:
    circuit.append(cirq.H(q))
    circuit.append(cirq.X(q))
circuit.append(cirq.H(A[-1]))
circuit.append(cirq.CNOT(A[0], A[1]))
circuit.append(cirq.H(A[-1]))
for q in A:
    circuit.append(cirq.X(q))
    circuit.append(cirq.H(q))

# Ölçüm
circuit.append([cirq.measure(q, key=f'a{i}') for i, q in enumerate(A)])
circuit.append([cirq.measure(q, key=f'b{i}') for i, q in enumerate(B)])

simulator = cirq.Simulator()
result = simulator.run(circuit, repetitions=10)
print(result)


a0=1100001100
a1=1001010001
b0=1110010001
b1=0110011100


Qiskit Örneği (2 qubit Grover + dolanıklık)

In [19]:
from qiskit import QuantumCircuit, Aer, execute

qc = QuantumCircuit(4,4)
A = [0,1]
B = [2,3]

# Başlangıç ve dolanıklık
for i in range(2):
    qc.h(A[i])
    qc.h(B[i])
    qc.cx(A[i], B[i])

# Oracle (aranan |00>)
qc.cz(A[0], A[1])

# Diffusion operator
for q in A:
    qc.h(q)
    qc.x(q)
qc.h(A[1])
qc.cx(A[0], A[1])
qc.h(A[1])
for q in A:
    qc.x(q)
    qc.h(q)

# Ölçüm
for i in range(2):
    qc.measure(A[i],i)
    qc.measure(B[i],i+2)

backend = Aer.get_backend('qasm_simulator')
result = execute(qc, backend, shots=10).result()
print(result.get_counts())


ImportError: cannot import name 'Aer' from 'qiskit' (/usr/local/lib/python3.12/dist-packages/qiskit/__init__.py)

QuTiP Örneği (2 qubit Grover + dolanıklık)

In [20]:
from qutip import *
import numpy as np

# Qubit sayısı
n = 2

# Başlangıç durumu |00>_A |00>_B
A = [basis(2,0) for _ in range(n)]
B = [basis(2,0) for _ in range(n)]
state = tensor(A+B)

# Hadamard A ve B
H = (1/np.sqrt(2)) * Qobj([[1,1],[1,-1]])
for i in range(n):
    H_i = tensor([H if j==i or j==i+n else qeye(2) for j in range(2*n)])
    state = H_i*state
    # Dolanıklık: CNOT
    X = sigmax()
    P0 = basis(2,0)*basis(2,0).dag()
    P1 = basis(2,1)*basis(2,1).dag()
    op = tensor([P0 if j==i else qeye(2) for j in range(2*n)]) + \
         tensor([P1 if j==i else (X if j==i+n else qeye(2)) for j in range(2*n)])
    state = op*state

# Basit oracle (|00> hedef)
Z = sigmaz()
oracle = tensor([Z if i==0 else qeye(2) for i in range(2*n)])
state = oracle*state

print(state)


Quantum object: dims=[[2, 2, 2, 2], [1]], shape=(16, 1), type='ket', dtype=Dense
Qobj data =
[[ 0.25]
 [ 0.25]
 [ 0.25]
 [ 0.25]
 [ 0.25]
 [ 0.25]
 [ 0.25]
 [ 0.25]
 [-0.25]
 [-0.25]
 [-0.25]
 [-0.25]
 [-0.25]
 [-0.25]
 [-0.25]
 [-0.25]]
